In [1]:
import ee
import pandas as pd

# Authenticate and initialize GEE
ee.Initialize(project="micropower-app")

# Sample coordinates for debugging
data = {
    "latitude": [14.5995, 10.3157, 16.4023],  # Manila, Cebu, Baguio
    "longitude": [120.9842, 123.8854, 120.5960]
}
df = pd.DataFrame(data)

In [3]:

# Define datasets from ERA5
solar_radiation = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").select("surface_solar_radiation_downwards_max")
surface_temp = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").select("temperature_2m_max")
precipitation = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").select("total_precipitation_sum")

# Get the most recent available image
solar_latest = solar_radiation.sort("system:time_start", False).first()
temp_latest = surface_temp.sort("system:time_start", False).first()
precip_latest = precipitation.sort("system:time_start", False).first()

def extract_values(lat, lon):
    """Fetches feature values from GEE for a given coordinate."""
    point = ee.Geometry.Point(lon, lat)
    
    try:
        solar_val = solar_latest.sample(region=point, scale=1000).first().get("surface_solar_radiation_downwards_max").getInfo()
        temp_val = temp_latest.sample(region=point, scale=1000).first().get("temperature_2m_max").getInfo()
        precip_val = precip_latest.sample(region=point, scale=1000).first().get("total_precipitation_sum").getInfo()
    except Exception as e:
        print(f"Error fetching values for ({lat}, {lon}): {e}")
        return None, None, None
    
    return solar_val, temp_val, precip_val

# Apply function to DataFrame
df[["Solar Radiation", "Max Temp", "Precipitation"]] = df.apply(lambda row: extract_values(row["latitude"], row["longitude"]), axis=1, result_type="expand")

# Display DataFrame
df


Error fetching values for (10.3157, 123.8854): Element.get: Parameter 'object' is required and may not be null.


,latitude,longitude,Solar Radiation,Max Temp,Precipitation
0,14.5995,120.9842,2498662.0,303.106339,0.000339
1,10.3157,123.8854,NaN,NaN,NaN
2,16.4023,120.5960,2712412.0,296.911545,0.000802
